In [90]:
!pip install joblib


descomposición de imagenes  para que el modelo funcione con los colores carácterisitcos de cada imagen y asi poder realizar su trainig y despues proseguir con su test. Para terminar el modelo s eguarda en un pkl, par apoder usarlo despues.

In [99]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import joblib

def extract_colors(image, num_colors=4):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    kmeans = KMeans(n_clusters=num_colors)
    kmeans.fit(image)

    colors = kmeans.cluster_centers_.astype(int)
    return colors.flatten()

def load_data(base_path):
    X, y = [], []
    for season in os.listdir(base_path):
        season_path = os.path.join(base_path, season)
        if os.path.isdir(season_path):
            for img_name in os.listdir(season_path):
                img_path = os.path.join(season_path, img_name)
                image = cv2.imread(img_path)
                if image is not None:
                    image = cv2.resize(image, (100, 100))
                    colors = extract_colors(image, num_colors=4)
                    X.append(colors)
                    y.append(season)
    return np.array(X), np.array(y)

base_path = 'train'
X, y = load_data(base_path)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
joblib.dump(model, 'model_colorimetria.pkl')


              precision    recall  f1-score   support

    invierno       0.44      0.67      0.53         6
       otono       0.50      0.40      0.44         5
   primavera       1.00      0.33      0.50         6
      verano       0.20      0.33      0.25         3

    accuracy                           0.45        20
   macro avg       0.54      0.43      0.43        20
weighted avg       0.59      0.45      0.46        20



['model_colorimetria.pkl']

de est forma se realiza una pequeña muestra del modelo, obtendieno adicionalemnte los colores mas recomendados dependiendo del tipo de colorimetría que posee.

In [113]:
import joblib
import cv2
import numpy as np
import matplotlib.pyplot as plt

def extract_colors(image, num_colors=4):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    kmeans = KMeans(n_clusters=num_colors, random_state=42)
    kmeans.fit(image)

    colors = kmeans.cluster_centers_.astype(int)
    return colors.flatten()

def plot_colors(image):
    colors = extract_colors(image)
    colors = colors.reshape(-1, 1, 3)
    plt.figure(figsize=(8, 6))
    plt.imshow(colors.astype(int))
    plt.axis('off')
    plt.show()

def predict_season(image, model):
    colors = extract_colors(image, num_colors=4)
    colors = colors.reshape(1, -1)
    prediction = model.predict(colors)
    return prediction[0]

def get_hex_colors(season):
    colors = {
        'invierno': ['#000000', '#FFFFFF', '#FF00FF', '#8B0000', '#00FFFF', '#C0C0C0'],  # Negro, blanco, fucsia, rojo oscuro, cian, plata
        'verano': ['#FFD700', '#FF69B4', '#87CEEB', '#D3D3D3', '#00BFFF', '#F0E68C'],    # Amarillo pálido, fucsia, azul cielo, gris claro, azul profundo, caqui
        'primavera': ['#FF4500', '#FFDAB9', '#FFFF00', '#7FFFD4', '#FF6347', '#98FB98'],  # Rojo anaranjado, melocotón, amarillo, aqua, tomate, verde claro
        'otoño': ['#8B4513', '#CD5C5C', '#FFA500', '#FFE4B5', '#A0522D', '#8FBC8F']        # Marrón, rojo, naranja, beige, marrón claro, verde mar
    }
    return colors.get(season.lower(), [])

model = joblib.load('model_colorimetria.pkl')

example_image = cv2.imread('nuevo12.jpg')
if example_image is not None:
   # for _ in range(5):
        predicted_season = predict_season(example_image, model)
        print(f"La imagen es de la estación: {predicted_season}")
        hex_colors = get_hex_colors(predicted_season)
        print(f"Colores en formato hexadecimal para {predicted_season}: {hex_colors}")
  #  plot_colors(example_image)
else:
    print("Error al cargar la imagen.")


La imagen es de la estación: invierno
Colores en formato hexadecimal para invierno: ['#000000', '#FFFFFF', '#FF00FF', '#8B0000', '#00FFFF', '#C0C0C0']
